In [1]:
import pandas as pd
from pathlib import Path

In [2]:
def process_energy_file(filepath):
    # Read the raw Excel file
    df_raw = pd.read_excel(filepath, header=None)
    df_raw = df_raw.drop(index=[0, 2, 3, 53, 54, 55, 56, 57, 58])
    df_raw.reset_index(drop=True, inplace=True)

    label_row = df_raw.iloc[1].copy()
    df_raw.drop(index=1, inplace=True)
    df_raw.reset_index(drop=True, inplace=True)

    # Find indices for the "System Demand (Actual)" columns
    keep_cols = [0]  # keep first col
    for i, val in enumerate(label_row):
        if isinstance(val, str) and "System Demand" in val:
            keep_cols.append(i)

    df_raw = df_raw.iloc[:, keep_cols]

    df_raw.columns = df_raw.iloc[0]  # first row as header
    df_raw = df_raw[1:]  # drop the header row

    # Melt
    df_long = df_raw.melt(id_vars='Date', var_name='DateColumn', value_name='Demand')
    df_long.rename(columns={'Date': 'Time', 'DateColumn': 'Day'}, inplace=True)

    df_long['Date'] = pd.to_datetime(df_long['Day'].astype(str) + ' ' + df_long['Time'].astype(str))
    df = df_long.sort_values(by="Date")
    df = df[["Date", "Demand"]]
    df.reset_index(drop=True, inplace=True)

    return df

In [3]:
data_dir = Path("data/energy_usage")
# Get all weekly Excel files in a list
files = sorted(data_dir.glob("*.xls"))

In [4]:
all_data = pd.concat([process_energy_file(f) for f in files], ignore_index=True)

C:\Users\rocky\AppData\Local\Temp\ipykernel_14412\1881950864.py:26: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_long['Date'] = pd.to_datetime(df_long['Day'].astype(str) + ' ' + df_long['Time'].astype(str))
C:\Users\rocky\AppData\Local\Temp\ipykernel_14412\1881950864.py:26: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_long['Date'] = pd.to_datetime(df_long['Day'].astype(str) + ' ' + df_long['Time'].astype(str))
C:\Users\rocky\AppData\Local\Temp\ipykernel_14412\1881950864.py:26: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_long['Date'] = pd.to_datetime(df_long['Da

In [5]:
all_data.to_csv("demand_data.csv", index=None)